<a href="https://colab.research.google.com/github/rafaelportomoura/ufla-gcc128-inteligencia-artificial/blob/main/Knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho – Implementação de KNN - Método de Classificação para uma data set ( Iris ou Spam)


## Background

### KNN

O algoritmo K-Nearest Neighbors (KNN) é um método de aprendizado de máquina supervisionado utilizado para classificação e regressão. Ele opera com base na ideia de que itens semelhantes tendem a estar próximos uns dos outros. Para classificar um novo ponto de dados, o KNN calcula a distância entre esse ponto e os pontos de treinamento conhecidos e seleciona os K vizinhos mais próximos.

A classe ou valor alvo do novo ponto é determinado pela maioria das classes (no caso de classificação) ou pela média dos valores (no caso de regressão) dos K vizinhos mais próximos. O valor de K, que representa o número de vizinhos considerados, é um hiperparâmetro que pode ser ajustado para otimizar o desempenho do modelo.

A distância utilizada como métrica é a distância euclidiana, representada pela equação abaixo, onde $p$ e $p'$ são dois objetoros representados por vetores no espaço e n é a quantidade de dimensões.


$$ d(p,q) = \sqrt{\sum_{i=1}^{n}{(p - q)^2}} $$

### Data Set Iris

O conjunto de dados Iris é um dos conjuntos mais icônicos e amplamente utilizados em aprendizado de máquina e estatísticas. Ele contém informações sobre três espécies diferentes de plantas de íris: Setosa, Versicolor e Virginica. Para cada uma das 150 amostras de íris, são registradas quatro características: comprimento e largura das sépalas e pétalas. Esse conjunto de dados é frequentemente utilizado para tarefas de classificação e clustering, bem como para ilustrar conceitos fundamentais em análise de dados e visualização. Graças à sua simplicidade e clareza, o conjunto de dados Iris é uma escolha popular para iniciantes em aprendizado de máquina, bem como para pesquisadores que desejam explorar algoritmos e técnicas de análise de dados. O conjunto de dados pode ser acessado no repositório UCI Machine Learning e é uma valiosa ferramenta para estudos e experimentos na área de ciência de dados.

[Página do dataset](https://archive.ics.uci.edu/dataset/53/iris)

## Código

#### Cabeçalho

In [8]:
pip install scikit-learn

In [6]:
from sklearn.datasets import  load_iris
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
import math
from google.colab import files
import io
import pandas as pd
from tabulate import tabulate

Logger


In [18]:
class Logger:
    def __init__(self,log_type: str):
        self.log_type = log_type if log_type else 'standard'
        self.__error__ = log_type in ['debug','info','standard', 'error']
        self.__info__ = log_type in ['debug','info','standard']
        self.__debug_logs__ = log_type in ['debug','standard']
        self.__log__ = log_type in ['debug','info','standard','log','error']

    def debug(self,*args):
        if(self.__debug_logs__):
            print(*args)
    def info(self,*args):
        if(self.__info__):
            print(*args)
    def error(self,*args):
        if(self.__error__):
            print(*args)
    def log(self,*args):
        if(self.__log__):
            print(*args)

logger = Logger('log')

### Algoritmo KNN


Distância Euclidiana


In [20]:
def distancia_euclidiana(p,q):
    subtraidos = np.subtract(p,q)
    elevado = np.power(subtraidos,2)
    somatorio = np.sum(elevado)
    distancia = math.sqrt(somatorio)
    return distancia

Ponto

In [28]:
class Ponto:
    def __init__(self, valores: list[int], classe: int = None):
        self.valores = valores
        self.classe = classe
        self.vizinhos = []

    def distancia(self, valores_de_outro_ponto: list[int],) -> float:
        return distancia_euclidiana(self.valores, valores_de_outro_ponto)

    def calcula_vizinhos_mais_proximos(self, outros_pontos, numero_maximo_de_vizinhos: int) -> None:
        for q in outros_pontos:
            self.calcula_distancia_e_adiciona_aos_vizinhos(q,numero_maximo_de_vizinhos)

    def calcula_distancia_e_adiciona_aos_vizinhos(self, outro_ponto, numero_maximo_de_vizinhos: int) -> bool:
        distancia = self.distancia(outro_ponto.valores)
        if (len(self.vizinhos) < numero_maximo_de_vizinhos):
            self.vizinhos.append([outro_ponto,distancia])
            self.vizinhos.sort(key=lambda i: i[1])
            return True

        if distancia < self.vizinhos[-1][1]:
            self.vizinhos.pop()
            self.vizinhos.append([outro_ponto,distancia])
            self.vizinhos.sort(key=lambda i: i[1])
            return True

        return False

    def define_classe(self) -> int:
        frequencia = {}
        menor_distancia_por_classe = {}
        classes = [vizinho[0].classe for vizinho in self.vizinhos]
        conjunto_de_classes = set(classes)

        frequencia_maxima = 0
        for classe in conjunto_de_classes:
            index_de_classes = [i for i in classes if i == classe]
            frequencia[classe] = len(index_de_classes)
            if frequencia[classe] > frequencia_maxima:
                frequencia_maxima = frequencia[classe]
            menor_distancia_por_classe[classe] = min(self.vizinhos, key=lambda v: v[1])[1]

        moda = [classe for classe in frequencia if frequencia[classe] == frequencia_maxima]

        logger.debug(f"""
        DefineClasse
            frequencia: {frequencia}
            frequencia_maxima: {frequencia_maxima}
            moda: {moda}
            menor_distancia_por_classe: {menor_distancia_por_classe}
        """)

        if len(moda) == 1:
            self.classe = moda[0]
        else:
            menor_distancia_das_modas = float('inf')
            self.classe = [x for x in moda if menor_distancia_por_classe[x] < menor_distancia_das_modas][0]

        logger.debug(f'classe: {self.classe}')
        return self.classe





Orquestrador

In [22]:
class KNN:
    def __init__(self,vizinhos: int):
        self.altera_quantidade_de_vizinhos(vizinhos)
        self.pontos_de_treinamento = []
        self.dimensoes = 1
        self.total_de_valores_treinados = 0

    def treinar(self, valores: list, classes: list) -> None:
        self.total_de_valores_treinados = len(valores)
        self.dimensoes = len(valores[0])
        self.pontos_de_treinamento = [Ponto(valores[i], classes[i]) for i in range(0,self.total_de_valores_treinados)]
        logger.debug(f"""KNN
        dimensões: {self.dimensoes}
        total: {self.total_de_valores_treinados}
        """)

    def altera_quantidade_de_vizinhos(self,vizinhos: int) -> None:
        if vizinhos == 0:
            raise Exception('Vizinhos deve ser maior do que 0!')
        self.quantidade_de_vizinhos = vizinhos

    def classificar(self, valores: list) -> list[int]:
        if len(self.pontos_de_treinamento) == 0:
            raise Exception('É necessário treinar antes!')
        pontos = []
        classes = []
        for v in valores:
            logger.debug(f"valor: {v}")
            ponto = Ponto(v)
            ponto.calcula_vizinhos_mais_proximos(self.pontos_de_treinamento, self.quantidade_de_vizinhos)
            classe = ponto.define_classe()
            logger.debug(f"\t{classe}")
            classes.append(classe)

        return classes


    def pontuar(self, classificados: list, classes: list) -> float:
        if len(classificados) != len(classes):
            raise Exception('O tamanho dos classificados está diferente do tamanho das classes!')

        total = len(classificados)
        corretos = len([i for i in range(0,total) if classes[i] == classificados[i]])

        return corretos/total

    def matriz_de_confusao(self, classificados: list, classes: list, conjunto_de_classes: dict = None) -> list[list[int]]:
        if len(classificados) != len(classes):
            raise Exception('O tamanho dos classificados está diferente do tamanho das classes!')

        matriz_confusao = []
        cabecalho = []
        if not conjunto_de_classes:
            conjunto_de_classes = list(set([*classes,*classificados]))
            cabecalho = conjunto_de_classes.copy()
        else:
            cabecalho = list(conjunto_de_classes.values()).copy()
        matriz_confusao.append(['',*cabecalho])

        i = 0
        for classe in conjunto_de_classes:
            indexes_de_classe = [x for x in range(0,len(classes)) if classes[x] == classe]
            frequencia = len(indexes_de_classe)
            linha = [cabecalho[i]]
            i+= 1
            for x in conjunto_de_classes:
                if (frequencia == 0):
                    frequencia = 1
                linha.append(f'{(len([y for y in indexes_de_classe if classificados[y] == x]) * 100/frequencia):.2f}')
            matriz_confusao.append(linha)
        return matriz_confusao



In [23]:
def rodar_knn(
    vizinhos: int,
    valores_de_treinamento: list[int],
    classes_de_treinamento: list[int],
    valores_de_teste: list[int],
    classes_de_teste: list[int],
    conjunto_de_classes: dict = None
):
    knn = KNN(vizinhos)
    knn.treinar(valores_de_treinamento, classes_de_treinamento)
    classificados = knn.classificar(valores_de_teste)
    pontuacao = knn.pontuar(classificados, classes_de_teste)
    matriz_confusao = knn.matriz_de_confusao(classificados, classes_de_teste,conjunto_de_classes)
    tabela_confusao = tabulate(matriz_confusao, headers="firstrow", tablefmt="grid")
    logger.log(f'🌇 Vizinhos: {vizinhos}')
    logger.log(f'🥇 Pontuação: {(pontuacao * 100):.2f}%')
    logger.log(tabela_confusao)
    logger.log(f'🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚')

### Dataset Íris

Carregamento do dataset

In [24]:
IRIS_VALORES, IRIS_CLASSES = load_iris(return_X_y=True)

Roda KNN com Iris

In [25]:
IRIS_TAMANHO_DO_TESTE = 0.7
VALORES_DE_TREINAMENTO, VALORES_DE_TESTE, CLASSES_DE_TREINAMENTO, CLASSES_DE_TESTE = train_test_split(IRIS_VALORES, IRIS_CLASSES, test_size=IRIS_TAMANHO_DO_TESTE, random_state=3)
serie_de_vizinhos = 10
tamanho_do_teste = -1
for vizinhos in range(1,serie_de_vizinhos+1):
    rodar_knn(
        vizinhos,
        VALORES_DE_TREINAMENTO,
        CLASSES_DE_TREINAMENTO,
        VALORES_DE_TESTE[:tamanho_do_teste],
        CLASSES_DE_TESTE[:tamanho_do_teste],
        {0: 'Setosa', 1: 'Versicolor', 2: 'Virginica'}
    )

🌇 Vizinhos: 1
🥇 Pontuação: 95.19%
+------------+----------+--------------+-------------+
|            |   Setosa |   Versicolor |   Virginica |
+============+==========+==============+=============+
| Setosa     |      100 |         0    |        0    |
+------------+----------+--------------+-------------+
| Versicolor |        0 |        85.29 |       14.71 |
+------------+----------+--------------+-------------+
| Virginica  |        0 |         0    |      100    |
+------------+----------+--------------+-------------+
🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚
🌇 Vizinhos: 2
🥇 Pontuação: 95.19%
+------------+----------+--------------+-------------+
|            |   Setosa |   Versicolor |   Virginica |
+============+==========+==============+=============+
| Setosa     |      100 |         0    |        0    |
+------------+----------+--------------+-------------+
| Versicolor |        0 |        88.24 |       11.76 |
+------------+----------+--------------+-------------+
| Virginica  |        0 |    

### Dataset Spam

Upload File

In [11]:
uploaded = files.upload()
SPAM_DATASET = pd.read_csv(io.BytesIO(uploaded['spambase.data']))
SPAM_DATASET.head()

Saving spambase.data to spambase.data


,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,class
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


Converte os dados

In [26]:
SPAM = 1
NOT_SPAM = 0
def get_spam_dataset(tamanho_do_teste: float, spam_dataset=SPAM_DATASET) -> list[list,list,list,list]:
    classes = list(spam_dataset['class']).copy()
    spam_dataset.drop(columns='class')
    valores = spam_dataset.values.tolist()

    classes_divididas = np.array_split(classes,4)
    valores_divididos = np.array_split(valores,4)

    classes_de_teste = []
    classes_de_treinamento = []
    valores_de_teste = []
    valores_de_treinamento = []

    for classe_dividido in classes_divididas:
        tamanho = len(classe_dividido)
        tamanho_de_teste = int(tamanho_do_teste * tamanho)
        classes_de_teste.extend(classe_dividido[:tamanho_de_teste])
        classes_de_treinamento.extend(classe_dividido[tamanho_de_teste:])

    for valor_dividido in valores_divididos:
        tamanho = len(valor_dividido)
        tamanho_de_teste = int(tamanho_do_teste * tamanho)
        valores_de_teste.extend(valor_dividido[:tamanho_de_teste])
        valores_de_treinamento.extend(valor_dividido[tamanho_de_teste:])

    return valores_de_treinamento, classes_de_treinamento, valores_de_teste, classes_de_teste


##### Run

In [30]:
serie_de_vizinhos = 10
tamanho_do_teste = 0.7
pra_testar =  -1
VALORES_DE_TREINAMENTO, CLASSES_DE_TREINAMENTO, VALORES_DE_TESTE, CLASSES_DE_TESTE = get_spam_dataset(tamanho_do_teste)
for vizinhos in range(1,serie_de_vizinhos+1):
    rodar_knn(
        vizinhos,
        VALORES_DE_TREINAMENTO,
        CLASSES_DE_TREINAMENTO,
        VALORES_DE_TESTE[:pra_testar],
        CLASSES_DE_TESTE[:pra_testar],
        {SPAM: 'SPAM',NOT_SPAM: 'NÃO SPAM'}
    )

🌇 Vizinhos: 1
🥇 Pontuação: 70.92%
+----------+--------+------------+
|          |   SPAM |   NÃO SPAM |
+==========+========+============+
| SPAM     |  49.49 |      50.51 |
+----------+--------+------------+
| NÃO SPAM |  11.13 |      88.87 |
+----------+--------+------------+
🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚
🌇 Vizinhos: 2
🥇 Pontuação: 66.98%
+----------+--------+------------+
|          |   SPAM |   NÃO SPAM |
+==========+========+============+
| SPAM     |  34.56 |      65.44 |
+----------+--------+------------+
| NÃO SPAM |   5.88 |      94.12 |
+----------+--------+------------+
🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚
🌇 Vizinhos: 3
🥇 Pontuação: 69.84%
+----------+--------+------------+
|          |   SPAM |   NÃO SPAM |
+==========+========+============+
| SPAM     |  46.9  |      53.1  |
+----------+--------+------------+
| NÃO SPAM |  10.96 |      89.04 |
+----------+--------+------------+
🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚🔚
🌇 Vizinhos: 4
🥇 Pontuação: 67.85%
+----------+--------+------------+
|          |   SPAM |   N